References to: 
https://www.kaggle.com/code/tj00001/building-music-recommendation-system-using-spotify#Read-Data

https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

In [1]:
pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [13]:
cp kaggle.json ~/.kaggle/

In [14]:
! chmod 600 ~/.kaggle/kaggle.json

In [15]:
! kaggle datasets download -d vatsalmavani/spotify-dataset

 55% 9.00M/16.5M [00:00<00:00, 35.0MB/s]
100% 16.5M/16.5M [00:00<00:00, 60.0MB/s]


In [16]:
! kaggle datasets download -d andrewmvd/spotify-playlists

 97% 177M/183M [00:02<00:00, 94.6MB/s]
100% 183M/183M [00:02<00:00, 89.5MB/s]


In [17]:
! unzip spotify-dataset.zip

Archive:  spotify-dataset.zip
  inflating: data/data.csv           
  inflating: data/data_by_artist.csv  
  inflating: data/data_by_genres.csv  
  inflating: data/data_by_year.csv   
  inflating: data/data_w_genres.csv  


In [18]:
! unzip spotify-playlists.zip

Archive:  spotify-playlists.zip
  inflating: README.txt              
  inflating: spotify_dataset.csv     


In [9]:
pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879168 sha256=5a896fa15d23ace40f9d462cb7ee0d1743618bff76751c82b28e88b01d993d33
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [19]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
import ast
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from time import time
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
data = pd.read_csv("./data/data.csv")
genre_data = pd.read_csv('./data/data_by_genres.csv')
year_data = pd.read_csv('./data/data_by_year.csv')

In [21]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [22]:
print(genre_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mode              2973 non-null   int64  
 1   genres            2973 non-null   object 
 2   acousticness      2973 non-null   float64
 3   danceability      2973 non-null   float64
 4   duration_ms       2973 non-null   float64
 5   energy            2973 non-null   float64
 6   instrumentalness  2973 non-null   float64
 7   liveness          2973 non-null   float64
 8   loudness          2973 non-null   float64
 9   speechiness       2973 non-null   float64
 10  tempo             2973 non-null   float64
 11  valence           2973 non-null   float64
 12  popularity        2973 non-null   float64
 13  key               2973 non-null   int64  
dtypes: float64(11), int64(2), object(1)
memory usage: 325.3+ KB
None


In [23]:
print(year_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mode              100 non-null    int64  
 1   year              100 non-null    int64  
 2   acousticness      100 non-null    float64
 3   danceability      100 non-null    float64
 4   duration_ms       100 non-null    float64
 5   energy            100 non-null    float64
 6   instrumentalness  100 non-null    float64
 7   liveness          100 non-null    float64
 8   loudness          100 non-null    float64
 9   speechiness       100 non-null    float64
 10  tempo             100 non-null    float64
 11  valence           100 non-null    float64
 12  popularity        100 non-null    float64
 13  key               100 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 11.1 KB
None


In [24]:
data['artists'] = data['artists'].map(lambda x: x.lstrip('[').rstrip(']'))
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"'Sergei Rachmaninoff', 'James Levine', 'Berlin...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,'Dennis Day',0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,'KHP Kridhamardawa Karaton Ngayogyakarta Hadin...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,'Frank Parker',0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,'Phil Regan',0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [25]:
p = 0.02  # to randomly select 1% of the rows
df_playlist = pd.read_csv('./spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_playlist.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,Out Of My Mind,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Oasis,Little By Little,IOW 2012
2,07f0fc3be95dcd878966b1f9572ff670,Bonobo,Prelude - Lapalux's Finger on the Tape Remix,Chill out
3,07f0fc3be95dcd878966b1f9572ff670,Zero 7,You're My Flame,Chill out
4,07f0fc3be95dcd878966b1f9572ff670,Daft Punk,Emotion,Daft Punk


In [26]:
len(df_playlist)

257967

In [27]:
df_playlist.columns = df_playlist.columns.str.replace('"', '')
df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

Only keeping artists with frequency higher than 50 

In [28]:
df_playlist = df_playlist.groupby('artist').filter(lambda x : len(x)>=50)

In [29]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]

In [30]:
size = lambda x: len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
6729,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,72
11066,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,53
30918,b1d4116e7cf150ae7d77413620f5f571,Wolfgang Amadeus Mozart,45
43826,fa849dabeb14a2800ad5130907fc5018,Peggy Lee,45
43805,fa849dabeb14a2800ad5130907fc5018,Louis Armstrong,37


In [31]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Wolfgang Amadeus Mozart
2,2,Peggy Lee
3,3,Louis Armstrong
4,4,Ella Fitzgerald


In [32]:
def GetInPut(user):
    inputArtist = pd.DataFrame(user)
    #Filtering out the movies by title
    Id = df_artist[df_artist['artist'].isin(inputArtist['artist'].tolist())]
    #Then merging it so we can get the movieId. It's implicitly merging it by title.
    inputArtist = pd.merge(Id, inputArtist)
    #Dropping information we won't use from the input dataframe
    #inputArtist = inputArtist.drop('year', 1)
    return inputArtist

In [33]:
user = [
            {'artist':'Ella Fitzgerald', 'freq':40},
            {'artist':'Frank Sinatra', 'freq':10},
            {'artist':'Lil Wayne', 'freq':3},
            {'artist':"The Rolling Stones", 'freq':5},
            {'artist':'Louis Armstrong', 'freq':5}
         ] 

In [34]:
inputArtist = GetInPut(user)

In [39]:
def ColFilter(inputArtist,df_freq):
    #Filtering out the artist by title
    Id = df_artist[df_artist['artist'].isin(inputArtist['artist'].tolist())]
    #Then merging it so we can get the artistId. It's implicitly merging it by title.
    inputArtist = pd.merge(Id, inputArtist)
    #Dropping information we won't use from the input dataframe
    #inputArtist = inputArtist.drop('year', 1)
    df_freq  = pd.merge(df_freq , df_artist, how='inner', on='artist')
    userSubsetGroup = df_freq.groupby(['user_id'])
    userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
    userSubsetGroup = userSubsetGroup[0:100]
    #Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
    pearsonCorDict = {}
    #For every user group in our subset
    for name, group in userSubsetGroup:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='artist_id')
        inputArtist = inputArtist.sort_values(by='artist_id')
        #Get the N for the formula
        n = len(group)
        #Get the review scores for the movies that they both have in common
        temp = inputArtist[inputArtist['artist_id'].isin(group['artist_id'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp['freq'].tolist()
        #put the current user group reviews in a list format
        tempGroupList = group['freq'].tolist()
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
        else:
            pearsonCorDict[name] = 0
    pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['user_id'] = pearsonDF.index
    pearsonDF.index = range(len(pearsonDF))
    topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
    topUsersRating=topUsers.merge(df_freq, left_on='user_id', right_on='user_id', how='inner')
    topUsersRatings=topUsersRating
    topUsersRating['weightedFreq'] = topUsersRating['similarityIndex']*topUsersRating['freq']
    #Applies a sum to the topUsers after grouping it up by userId
    tempTopUsersRating = topUsersRating.groupby('artist_id').sum()[['similarityIndex','weightedFreq']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedFreq']
    #Creates an empty dataframe
    recommendation_df = pd.DataFrame()
    #Now we take the weighted average
    recommendation_df['weighted average freq score'] = tempTopUsersRating['sum_weightedFreq']/tempTopUsersRating['sum_similarityIndex']
    recommendation_df['artist_id'] = tempTopUsersRating.index
    # Sort the recomendation by the weighted average freq score
    recommendation_df = recommendation_df.sort_values(by='weighted average freq score', ascending=False)
    recommendation_final = df_artist.loc[df_artist['artist_id'].isin(recommendation_df.head(10)['artist_id'].tolist())]
    return recommendation_final,topUsersRatings

In [40]:
recommendation_final, topUsersRatings =ColFilter(inputArtist,df_freq)
recommendation_final

,artist_id,artist
2,2,Peggy Lee
3,3,Louis Armstrong
4,4,Ella Fitzgerald
6,6,Frank Sinatra
8,8,B.B. King
10,10,Nat King Cole
17,17,Jamey Aebersold Play-A-Long
26,26,Boyce Avenue
55,55,Bad Religion
62,62,Rick Ross


In [41]:
df_playlist_2  = pd.merge(df_playlist , df_artist, how='inner', on='artist')
df_playlist_2.head()

,user_id,artist,track,playlist,artist_id
0,07f0fc3be95dcd878966b1f9572ff670,Bonobo,Prelude - Lapalux's Finger on the Tape Remix,Chill out,426
1,45624a435588a4e06de0ba38d3664e77,Bonobo,First Fires,Concentration 64,426
2,45624a435588a4e06de0ba38d3664e77,Bonobo,Transits,Concentration 64,426
3,f1ea49602239ca058d86ff0d3c1c792b,Bonobo,Days To Come - Instrumental,Bonobo,426
4,b38804ddef21178dd8a94dc897a2ea69,Bonobo,Pieces,zonin',426


In [42]:
data['song_id']=data.index
df = data[['danceability','energy',"valence","speechiness","instrumentalness","acousticness"]]
df.index = data['song_id']
# normalized  data by columns
df_normalized = pd.DataFrame(normalize(df, axis=1))
df_normalized.columns = df.columns
df_normalized.index = df.index
df_normalized.head()

,danceability,energy,valence,speechiness,instrumentalness,acousticness
song_id,,,,,,
0,0.204439,0.154611,0.043526,0.026819,0.643359,0.719566
1,0.526206,0.219092,0.618725,0.266637,0.000000,0.470308
2,0.238274,0.120590,0.028622,0.024627,0.663245,0.698114
3,0.258165,0.290084,0.154899,0.033233,0.000026,0.907802
4,0.382654,0.176680,0.231606,0.034787,0.000002,0.876076


In [45]:
"""
Music Recommender based on different distance calculation approaches

df_normalized: normalized song data
distance_method: distance calculation approach: e.g. cosine, euclidean, hamming
song_id: find similar songs based on the selected song
N: Top N song(s)

return 1) song data of selected song and Top N recommendation, 
2) song id and song name of Top N recommendation
"""

def Content_filter_music_recommender(song_id, N):
    distance_method = cosine
    allSongs = pd.DataFrame(df_normalized.index)
    allSongs = allSongs[allSongs.song_id != song_id]
    allSongs["distance"] = allSongs["song_id"].apply(lambda x: distance_method(df_normalized.loc[song_id], df_normalized.loc[x]))
    # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    TopNRecommendation = allSongs.sort_values(["distance"]).head(N).sort_values(by=['distance', 'song_id'])
    #print(data['name'].loc[song_id, :])
    Recommendation = pd.merge(TopNRecommendation , data, how='inner', on='song_id')
    SongName = Recommendation['name']
    return SongName

In [46]:
SongName=Content_filter_music_recommender(3, 5)
SongName

0                     Pause Track - Live
1    StaggerLee Has His Day at the Beach
2                            Pause Track
3                           Silent Track
4                           Magic Window
Name: name, dtype: object